In [1]:
import math
import faiss

from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

from langchain.chat_models import ChatOpenAI

from discussion_agents.core.base import BaseCore

In [2]:
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def relevance_score_fn(score: float) -> float:
    return 1.0 - score / math.sqrt(2)

def create_new_memory_retriever():
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

LLM = ChatOpenAI(openai_api_key=openai_api_key, max_tokens=1500)

core = BaseCore(
    llm=LLM,
    retriever=create_new_memory_retriever(),
)

In [4]:
instruction = "Describe what makes a table reliable."
name = "Bob"
age = 25
traits = "talkative, enthusiastic"
lifestyle = "lazy, likes to sleep late"
status = ""
_summary = "Bob is a socialite with lots of friends and a heart for enthusiasm and extrovertedness."

summary = (
    f"Name: {name}\n"
    + f"Age: {age}\n"
    + f"Innate traits: {traits}\n"
    + f"Status: {status}\n"
    + f"Lifestyle: {lifestyle}\n"
    + f"{_summary}\n"
)

In [5]:
from discussion_agents.planning.generative_agents import (
    generate_refined_plan_step
)

In [8]:
broad_plan = [
    "Choose a sturdy and durable material for the table, such as solid wood or metal",
    "Ensure that the table is properly constructed and assembled with strong joints and connections",
    "Verify that the table has a stable and level surface to prevent wobbling or tipping over",
    "Consider the weight capacity of the table to ensure it can support the intended load without any issues",
    "Regularly inspect and maintain the table, checking for any signs of wear, damage, or instability",
    "Avoid placing excessive weight or applying excessive force on the table to prevent potential damage",
    "Place the table on a suitable surface, such as a level floor, to provide additional stability",
    "Keep the table away from direct sunlight, extreme temperatures, or excessive moisture to prevent warping or deterioration",
    "Choose a table design that suits the intended purpose and environment, considering factors like size, shape, and functionality",
    "Consider purchasing a table from a reputable and trustworthy manufacturer or retailer to ensure quality and reliability",
]

refined_steps = generate_refined_plan_step(
    instruction="Something",
    previous_steps=["Something"],
    plan_step="Something",
    summary=summary,
    core=core,
    k=1
)
refined_steps

['<NO_SUBSTEPS_REQUIRED>']